
#  Statistics & Data Analysis


##  Req

#### Import Requirements

##### HTML formatting

In [1]:
import numpy as np
import pandas as z
import scipy
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype
from plotnine import *
from scipy.stats import *
import scikit_posthocs   as sp



data = pd.read_csv("./NewCols.csv")



ModuleNotFoundError: No module named 'scikit_posthocs'

## Calculating the differences between the noremalized values. 

In [ ]:
data_control = data[data["treatment"] == "baseline"]
data_control.to_csv("./control.csv")
data_treatment = data[data["treatment"] == "intravenous LPS"]
data_control.to_csv("./lps.csv")

procData = data_treatment


procData['diff_AVAR2'] = (
    np.array(data_treatment["AVAR2"]) - np.array(data_control["AVAR2"])).tolist()
procData["diff_CVAR2"] = (
    np.array(data_treatment["CVAR2"]) - np.array(data_control["CVAR2"])).tolist()
procData["diff_AWT2"] = (np.array(data_treatment["AWT2"]) -
                         np.array(data_control["AWT2"])).tolist()
procData["diff_CWT2"] = (np.array(data_treatment["CWT2"]) -
                         np.array(data_control["CWT2"])).tolist()


procData["diff_total2"] = (
    np.array(data_treatment["total2"]) - np.array(data_control["total2"])).tolist()
procData["diff_totalA"] = (
    np.array(data_treatment["totalA"]) - np.array(data_control["totalA"])).tolist()
procData["diff_totalC"] = (
    np.array(data_treatment["totalC"]) - np.array(data_control["totalC"])).tolist()
procData["diff_totalWT"] = (np.array(
    data_treatment["totalWT"]) - np.array(data_control["totalWT"])).tolist()
procData["diff_totalVar"] = (np.array(
    data_treatment["totalVar"]) - np.array(data_control["totalVar"])).tolist()

procData.to_csv("./procData.csv")

In [ ]:
newDF=  data_control[["testGroup","tg2"]]
newDF


In [ ]:
newDF.rename(columns = {'testGroup':'c_tg','tg2':'c_tg2'},  inplace=True) 
newDF


In [ ]:
newDF.index = procData.index
procData= pd.concat([procData,newDF], axis=1)


#### Difference Table


In [ ]:

pd.set_option('display.max_rows', procData.shape[0]+1)

diff_data = procData.loc[ :,"diff_AVAR2":"diff_totalVar" ]
diff_data.to_csv("./diffData.csv")

In [ ]:
diff_data.describe()

In [ ]:
diff_data.var()


In [ ]:
diff_data.std()


In [ ]:
diff_data.skew()


In [ ]:
diff_data.kurtosis().tolist()

In [ ]:
diff_data.kurtosis()

## QQ Data for LPS



### summary Statistics 

#### Baseline - summary stats - summary stats

In [ ]:
baseline_summary = data_control.loc[:,'AWT2':'total2']

baseline_summary.describe()

#### Variance & STD DEv

In [ ]:
baseline_summary.var()

In [ ]:
baseline_summary.std()

#### skew

In [ ]:
scipy.stats.skew(baseline_summary).tolist()


#### Kurtosis

In [ ]:
scipy.stats.kurtosis(baseline_summary).tolist()


#### Intravenous LPS - summary stats

In [ ]:
LPS_summary = data_treatment.loc[:,'AWT2':'total2']
LPS_summary.describe()

#### Variance & STd DEv

In [ ]:
LPS_summary.var()


In [ ]:
LPS_summary.std()

#### Skew


In [ ]:
scipy.stats.skew(LPS_summary)

#### Kurtosis


In [ ]:
scipy.stats.kurtosis(LPS_summary)


## Graph Data - 

In [ ]:
from plotnine import *
ggplot(data, aes(x='treatment', y='AWT2') ) + geom_boxplot() + geom_jitter(data,aes(colour='treatment',shape='treatment'))

In [ ]:
a = 0.05

wilcoxon(data_control["AWT2"],data_treatment["AWT2"])


In [ ]:
ggplot(data, aes(x='treatment', y='CWT2') ) + geom_boxplot() + geom_jitter(data,aes(colour='treatment',shape='treatment'))


In [ ]:
a = 0.05

wilcoxon(data_control["CWT2"],data_treatment["CWT2"])


In [ ]:
ggplot(data, aes(x='treatment', y='AVAR2') ) + geom_boxplot() + geom_jitter(data,aes(colour='treatment',shape='treatment'))


In [ ]:
a = 0.05

wilcoxon(data_control["AVAR2"],data_treatment["AVAR2"])


In [ ]:
ggplot(data, aes(x='treatment', y='CVAR2') ) + geom_boxplot() + geom_jitter(data,aes(colour='treatment',shape='treatment'))


In [ ]:
a = 0.05

wilcoxon(data_control["CVAR2"],data_treatment["CVAR2"])


In [ ]:
removed_outliers = data.total2.between(data.total2.quantile(.05), data.total2.quantile(.95))
data_total= data[removed_outliers]
ggplot(data_total, aes(x='treatment',y="total2" ), ) + geom_boxplot(outlier_shape = "") + geom_jitter(data_total,aes(y="total2",colour='treatment',shape='treatment') )  + ggtitle("QQ Plot of IRAK-1 expression per  GbP") + xlab("Treatment") + ylab("Total IRAK-1 Levels per Gigabase pair") + ylim(data_total.total2.quantile(.05), data_total.total2.quantile(.95))

In [ ]:
a = 0.05

wilcoxon(diff_data["diff_total2"])


In [ ]:
removed_outliers_diffData = diff_data.diff_total2.between(diff_data.diff_total2.quantile(.05), diff_data.diff_total2.quantile(.95))
difftotalData=diff_data[removed_outliers_diffData]
ggplot(difftotalData, aes( x='0',y='diff_total2') ) + geom_boxplot() + geom_point(color="red") + ylim(difftotalData.diff_total2.quantile(.05), difftotalData.diff_total2.quantile(.95)) + ggtitle("QQ Plot of changes in IRAK-1 levels per Gbp") + xlab("Treatment") + ylab("Changes in  IRAK-1 Levels per Gigabase pair") 


In [ ]:
data_plot = data_treatment
controlData = data_control['total2']
controlData

In [ ]:
data_plot["ctrl_total2"]=controlData.to_list()
data_plot



In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(data_plot.total2.to_numpy().reshape((-1, 1)), data_plot.ctrl_total2)
r_sq= model.score(data_plot.total2.to_numpy().reshape((-1, 1)), data_plot.ctrl_total2)
print('coefficient of determination:', r_sq)
print('intercept:', model.intercept_)
print('slope:', model.coef_)

In [ ]:

ggplot(data_plot,aes(x='total2',y='ctrl_total2') )  + geom_point() + geom_smooth(method='lm')


In [ ]:

from sklearn import linear_model
lm = linear_model.LinearRegression()


In [ ]:
shapiro_test = shapiro(data_control['total2'])
shapiro_test

In [ ]:
shapiro_test = shapiro(data_treatment['total2'])
shapiro_test

In [ ]:
shapiro_test = shapiro(diff_data['diff_total2'])
shapiro_test

In [ ]:
ggplot(data, aes(x='treatment', y='totalVar') ) + geom_boxplot() + geom_jitter(data,aes(colour='treatment',shape='treatment'))


In [ ]:
a = 0.05

wilcoxon(diff_data["diff_totalVar"])


In [ ]:
ggplot(data, aes(x='treatment', y='totalWT') ) + geom_boxplot() + geom_jitter(data,aes(colour='treatment',shape='treatment'))


In [ ]:
a = 0.05

wilcoxon(diff_data["diff_totalWT"])


In [ ]:
ggplot(data, aes(x='treatment', y='totalA') ) + geom_boxplot() + geom_jitter(data,aes(colour='treatment',shape='treatment'))


In [ ]:
a = 0.05

wilcoxon(diff_data["diff_totalA"])


In [ ]:
ggplot(data, aes(x='treatment', y='totalC') ) + geom_boxplot() + geom_jitter(data,aes(colour='treatment',shape='treatment'))


In [ ]:
a = 0.05

wilcoxon(diff_data["diff_totalC"])


## Statistics

### Total 2 Comparison

#### Wilcoxon non-parametric

In [ ]:
a = 0.05

w, p = wilcoxon(data_control["total2"],data_treatment["total2"])
print(w, p)

In [ ]:
if (p < a):
    print("As P"+str(p)+" is less than a: "+str(a))
    print( "we reject the Null Hypothesis.")
    print(". There is significant difference betwween the groups")
else: 
    print("As P"+p+" is larger than a: "+str(a))
    print( "we FAIL TO reject the Null Hypothesis.")
    print(". There is NOT a significant difference betwween the groups")

#### Freidman's Anova

In [ ]:
sp.posthoc_nemenyi_friedman(diff_data)

Friedman Tes 

### other

In [ ]:
a = 0.05

w, p = wilcoxon((data_control["totalA"]/data_control["totalC"] ),(data_treatment["totalA"]/data_treatment["totalC"]))
print(w, p)

In [ ]:
a = 0.05

w, p = wilcoxon((data_control["AVAR2"]/data_control["CVAR2"] ),(data_treatment["AVAR2"]/data_treatment["CVAR2"]))
print(w, p)

In [ ]:
a = 0.05

w, p = wilcoxon((data_control["AWT2"]/data_control["CWT2"] ),(data_treatment["AWT2"]/data_treatment["CWT2"]))
print(w, p)

In [ ]:
ggplot()+geom_histogram(procData,aes(x="tg2"))

In [ ]:
ggplot()+geom_histogram(procData,aes(x="mutant"))

In [ ]:
ggplot()+geom_bar(procData,aes(x="spliceVariant",fill="mutant"))

In [ ]:
ggplot()+geom_col(procData,aes(x="spliceVariant",y="diff_totalA/diff_totalC",fill="mutant"))

In [ ]:
a = 0.05
diff_data = procData[(data["totalC"] > 0 ) & (data["totalA"] > 0 )]
ggplot()+geom_histogram(diff_data,aes(x="tg2"))

In [ ]:

w, p = wilcoxon((diff_data["totalC"] )/(diff_data["totalA"]))
print(w, p)

In [ ]:
a = 0.05

w, p = wilcoxon(data_control["total2"],data_treatment["total2"])
print(w, p)

2 graphs 

1. Do the Table
3. Black and white
3. Make sure its not sloppy
4. 

control, LPS & Difference.

correlation plot for each patient - total 2 & diff_total2

Look for A/C ratios 

ggplot(data_plot,aes(x='total2',y='ctrl_total2') )  + geom_point(colour) + geom_smooth(method='lm')
